In [ ]:
# coding: utf-8

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.cm import get_cmap
import scipy.optimize

# Input experimental data

In [ ]:
df1 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Heating', header = 0)
df2 = pd.read_excel('experimental_data.xlsx', sheet_name = 'Cooling', header = 0)

In [ ]:
m = 1e-2

In [ ]:
sigma_h = 0.1e-3

In [ ]:
df1['h_a, м'] = m * df1['h_a, см']
df1['h_b, м'] = m * df1['h_b, см']
df2['h_a, м'] = m * df2['h_a, см']
df2['h_b, м'] = m * df2['h_b, см']

In [ ]:
rho = 13600
g = 9.81
zero_kelvin = 273

In [ ]:
sigma_p = 2 * rho * g * sigma_h

In [ ]:
df1['P, Па'] = rho * g * (df1['h_b, м'] - df1['h_a, м'])
df2['P, Па'] = rho * g * (df2['h_b, м'] - df2['h_a, м'])

In [ ]:
df1['T, K'] = df1['T, C'] + zero_kelvin
df2['T, K'] = df2['T, C'] + zero_kelvin

In [ ]:
df1['1/T, 1/K'] = 1 / df1['T, K']
df2['1/T, 1/K'] = 1 / df2['T, K']
df1['ln(P)'] = np.log(df1['P, Па'])
df2['ln(P)'] = np.log(df2['P, Па'])

In [ ]:
df1

# Least Squares

In [ ]:
def lstsqb(x, y):
    return (x.multiply(y).mean() - x.mean() * y.mean()) / (x.multiply(x).mean() - x.mean() ** 2)

In [ ]:
def lstsqa(x, y):
    return y.mean() - lstsqb(x, y) * x.mean()

In [ ]:
def lstsqbsem(x, y):
    return 1 / np.sqrt(len(x)) * np.sqrt((y.multiply(y).mean() - y.mean() ** 2) / (x.multiply(x).mean() - x.mean() ** 2) - lstsqb(x, y) ** 2)

# Part 1

# Plots

In [ ]:
def plot_data(a, df, q, p):
    a.xaxis.set_major_locator(plt.MultipleLocator(5))
    a.yaxis.set_major_locator(plt.MultipleLocator(1000))
    a.xaxis.set_minor_locator(plt.MultipleLocator(2.5))
    a.yaxis.set_minor_locator(plt.MultipleLocator(200))
    plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    a.set_ylim(bottom = y_min, top = y_max)
    a.set_xlim(left = x_min, right = x_max)
    a.grid(visible = True, linestyle = '--')
    a.grid(visible = True, linestyle = '--', which = 'minor')
    x = np.linspace(x_min, x_max, 1000)
    a.scatter(df['T, K'], df['P, Па'])
    a.plot(x, np.exp(p[1] - mu * q / (R * x)))
    a.set_xlabel('$T, \degree K$')
    a.set_ylabel('$P, Па$')

In [ ]:
delta_x = 5

In [ ]:
x_max = max(df1.loc[:, 'T, K']) + delta_x
y_max = 17 / 16 * max(df1.loc[:, 'P, Па'])
x_min = min(df1.loc[:, 'T, K']) - delta_x
y_min = 15 / 16 * min(df1.loc[:, 'P, Па'])

In [ ]:
name = "Set1"
cmap = get_cmap(name) # type: matplotlib.colors.ListedColormap
colors = cmap.colors  # type: list

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 1. Зависимость $P(T)$ (Нагревание)')
plot_data(ax, df1, q1, p1)
fig.savefig('plots/plot1.pdf')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
fig.suptitle('Рис. 2. Зависимость $P(T)$ (Охлаждение)')
plot_data(ax, df2, q2, p2)
fig.savefig('plots/plot2.pdf')

In [ ]:
p1, v1 = np.polyfit(df1['1/T, 1/K'], df1['ln(P)'], deg = 1, cov = True)
p2, v2 = np.polyfit(df2['1/T, 1/K'], df2['ln(P)'], deg = 1, cov = True)
err1 = np.sqrt(np.diag(v1))
err2 = np.sqrt(np.diag(v2))

In [ ]:
x_max = 17 / 16 * max(df1.loc[:, '1/T, 1/K'])
y_max = 17 / 16 * max(df1.loc[:, 'ln(P)'])
x_min = 15 / 16 * min(df1.loc[:, '1/T, 1/K'])
y_min = 15 / 16 * min(df1.loc[:, 'ln(P)'])

In [ ]:
x = np.linspace(0, x_max, 1000)

In [ ]:
def plot_log(a, df, p, v, i):
    a.xaxis.set_major_locator(plt.MultipleLocator(0.1e-3))
    a.yaxis.set_major_locator(plt.MultipleLocator(1))
    a.xaxis.set_minor_locator(plt.MultipleLocator(0.2e-4))
    a.yaxis.set_minor_locator(plt.MultipleLocator(0.1))
    err = np.sqrt(np.diag(v))
    plt.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    a.set_ylim(bottom = y_min, top = y_max)
    a.set_xlim(left = x_min, right = x_max)
    a.grid(visible = True, linestyle = '--')
    a.grid(visible = True, linestyle = '--', which = 'minor')
    x = np.linspace(x_min, x_max, 1000)
    a.scatter(df['1/T, 1/K'], df['ln(P)'], color = colors[i])
    a.plot(x, x * p[0] + p[1], label = 'Линейная модель: $f(\ln{P}) = C + k / T$', color = colors[i])
    a.plot([], [], ' ', label = '$k = {k:.3f} \pm {dk:.3f}, C = {C:.3f} \pm {dC:.3f}$'.format(k = p[0], dk = err[0], C = p[1], dC = err[1]))
    a.legend()
    a.set_xlabel('$1/T, 1 / \degree K$')
    a.set_ylabel('$ln(P)$')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
plot_log(ax, df1, p1, v1, 0)
fig.suptitle('Рис. 3. Зависимость $P(1/T)$ в логарифмических координатах')
fig.savefig('plots/plot3.pdf')

In [ ]:
fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize=(11.7, 8.3))
plot_log(ax, df2, p2, v2, 1)
fig.suptitle('Рис. 4. Зависимость $P(1/T)$ в логарифмических координатах')
fig.savefig('plots/plot4.pdf')

In [ ]:
R = 8.31

In [ ]:
mu = 0.046

In [ ]:
q1 = (-1) * p1[0] * R / mu
q2 = (-1) * p2[0] * R / mu
sigma_q1 = (-1) * err1[0] / p1[0] * q1
sigma_q2 = (-1) * err2[0] / p2[0] * q2

In [ ]:
q2

In [ ]:
sigma_q2

# Results

In [ ]:
df1.to_excel('output1.xlsx')
df2.to_excel('output2.xlsx')